How to set up selenium and client?

In [1]:
var importer = require('../Core');

var client;
var getClient = () => {
    return importer.interpretAll([
        'webdriver client node',
        'decrypt password',
        'log in Google'
    ])
    .then(r => {
        var createWebdriverClient = eval(r[0].code);
        client = createWebdriverClient('localhost', 4444);
        eval(r[1].code);
        eval(r[2].code);
        return client;
    });
};
module.exports = getClient;
getClient;



[Function: getClient]

In [2]:
$$.async();
getClient()
    .then(c => $$.sendResult(c))
    .catch(e => $$.sendError(e));


Initializing webdriver on localhost
d6f40697-322e-4202-95bc-356c3ed0df3c


How to download google timeline data using webdriver?

In [3]:
var months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'];
var getGoogleTimeline = () => {
    console.log('Google: Logging timeline history');
    return client
        .url('https://www.google.com/maps/timeline')
        .loginGoogle()
        .waitUntil(() => client.getUrl()
                   .then(url => url.indexOf('timeline') > -1, 20000, ''))
        .catch(function (e) {
            console.log(e);
        })
        .pause(3000)
        .click('button[jsaction="select-today"]')
        .pause(1000);
}
if (typeof client.getGoogleTimeline == 'undefined') {
    client.addCommand('getGoogleTimeline', getGoogleTimeline);
}
module.exports = getGoogleTimeline;
getGoogleTimeline;


In [4]:
var readAllPages = () => {
    // TODO: add stop conditions
    var oneDay = 24*60*60*1000; // hours*minutes*seconds*milliseconds
    var firstDate = new Date(2016,8,21);
    var secondDate = new Date();
    var days = Math.round(Math.abs((firstDate.getTime() - secondDate.getTime())/(oneDay)));
    var result = client;
    dayKey = '';
    for (var c = 0; c < days; c++) {
        result = result.readTimelinePage()
            .click('.previous-date-range-button')
            .pause(1000)
    }
    return result;
}
if (typeof client.getGoogleTimeline == 'undefined') {
    client.addCommand('readAllPages', readAllPages);
}
module.exports = readAllPages;
readAllPages;


In [5]:
var readTimelinePage = () => {
    var day, lastTime, currDate, dayKey;
    return client.getText('.timeline-title')
        .then(function (text) {
            day = text;
            currDate = new Date(day);
            return client.getText('.timeline-subtitle');
        })
        .then(function (text) {
            if (text != '') {
                day = text;
                currDate = new Date(day);
            }
            var newKey = currDate.getDate() 
                + months[currDate.getMonth()] 
                + (currDate.getFullYear() + '').substr(2, 2);
            if (newKey != dayKey) {
                dayKey = newKey;
            }
        })
        .elements('.timeline-item ~ [jsinstance]')
        .then(function (els) {
            var result = client;
            for (var e = 0; e < els.value.length; e++) {
                result = result.readTimelineRow(day, dayKey, els.value[e]);
            }
            return result;
        })
        .catch(function (e) {
            if (e.type != 'NoSuchElement')
                console.log(e);
        })
}
if (typeof client.readTimelinePage == 'undefined') {
    client.addCommand('readTimelinePage', readTimelinePage);
}
module.exports = readTimelinePage;
readTimelinePage;


In [6]:
var timelineCache = {};
var readTimelineRow = (day, dayKey, elem) => {
    var title, text, start, end;
    return client.elementIdElement(elem.ELEMENT, '.place-visit-title')
        .then(function (el) {
            // Location items
            start = null;
            end = null;
            if (el.value) {
                return client.elementIdText(el.value.ELEMENT);
            }
            else {
                // Driving items
                return client
                    .elementIdElement(elem.ELEMENT, '.timeline-item')
                    .then(function (el) {
                        return client.elementIdAttribute(el.value.ELEMENT, 'data-segment-key');
                    })
                    .then(function (attr) {
                        if(attr.value) {
                            var timelineData = attr.value.split(':');
                            start = new Date(parseFloat(timelineData[1]));
                            end = new Date(parseFloat(timelineData[2]));
                        }
                        return client.elementIdElement(elem.ELEMENT, '.timeline-item-title-content');
                    })
                    .then(function (el) {
                        return client.elementIdText(el.value.ELEMENT);
                    });
            }
        })
        .then(function (el) {
            title = el.value;
            return client.elementIdElement(elem.ELEMENT, '.timeline-item-text');
        })
        .then(function (el) {
            if(el.value) {
                return client.elementIdText(el.value.ELEMENT);
            }
        })
        .then(function (el) {
            if(el) {
                text = el.value;
            }
            return client.elementIdElement(elem.ELEMENT, '.duration-text');
        })
        .then(function (el) {
            return client.elementIdText(el.value.ELEMENT);
        })
        .then(function (el) {
            var duration = el.value;
            start = start || new Date(day + ' ' + duration.split('-')[0]);
            end = end || new Date(day + ' ' + duration.split('-')[1]);
            var length = end.getTime() - start.getTime();
            var newRow = {
                type: 'timeline',
                timeline: dayKey,
                name: title,
                location: text,
                time: start,
                length: isNaN(length) ? 0 : length
            };
            if (typeof timelineCache[newRow.timeline] == 'undefined') {
                timelineCache[newRow.timeline] = [newRow];
            }
            else {
                timelineCache[newRow.timeline] = timelineCache[newRow.timeline]
                    .filter(function (i) {
                        return i.time != newRow.time;
                    })
                    .concat([newRow]);
            }
        })
        .catch(function (e) {
            console.log(e);
        });
}
if (typeof client.readTimelineRow == 'undefined') {
    client.addCommand('readTimelineRow', readTimelineRow);
}
module.exports = readTimelineRow;
readTimelineRow;


In [7]:
var fs = require('fs');
$$.async()
client.getGoogleTimeline()
    .then((r) => readAllPages())
    .then(() => Object.keys(timelineCache).forEach(r => fs.writeFileSync('/Users/briancullinan/Timeline/timeline-' + timelineCache[r][0].timeline + '.json', JSON.stringify(timelineCache[r], null, 4))))
    .catch(e => $$.sendError(e));
//    .then(() => client.readAllPages())



In [3]:
$$.async();
client.windowHandles()
.then(h => $$.sendResult(h))
.catch(e => $$.sendError(e));

In [5]:
var locationCache = {};
var dayKey; var count = 0;
$$.async();
importer.streamJson('/Users/briancullinan/Downloads/Takeout 5/Location History/Location History.json', [true, {emitPath: true}], (match) => {
    if(match.path[0] === 'locations') {
        var currDate = new Date(parseInt(match.value.timestampMs));
        var newKey = currDate.getDate() + months[currDate.getMonth()] + (currDate.getFullYear() + '').substr(2, 2);
        if (newKey != dayKey) {
            dayKey = newKey;
        }
        var newRow = Object.assign({
                time: currDate,
                type: 'location',
                location: dayKey,
        }, match.value);
        var cache = locationCache[newRow.location];
        if(typeof cache === 'undefined') {
            cache = locationCache[newRow.location] = [];
        }
        cache[cache.length] = newRow;
    }
})
    .then(() => Object.keys(locationCache).forEach(r => fs.writeFileSync('/Users/briancullinan/Timeline/location-' + locationCache[r][0].location + '.json', JSON.stringify(locationCache[r], null, 4))))
    .then(() => $$.sendResult(locationCache[dayKey]))
    .catch(e => $$.sendError(e));

